In [92]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px


In [93]:
# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')


In [277]:
app = dash.Dash(__name__)
dropdown_options=[ {'label': 'Yearly Statistics', 'value': 'Yearly Statistics'},
                    {'label': 'Recession Period Statistics', 'value': 'Recession Period Statistics'} ]
year_list = [i for i in range(1980, 2024, 1)]
app.layout = html.Div([
    #TASK 2.1 Add title to the dashboard
    html.H1("Automobile Sales Statistics Dashboard", style={'textAlign':'center','color':'#000000', 'font-size':24}),#May include style for title
    html.Div([#TASK 2.2: Add two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(id='stats', 
                   options=dropdown_options,
                   placeholder='',
                   style={'textAlign':'left','color':'#000000', 'font-size':14},
                   )]),  # value='Yearly Statistics' basically shows what is preselected , can be left out actually
    html.Div([
        html.Label("select-year"),
        dcc.Dropdown(
             id='select-year',
             options=[{'label': i, 'value': i} for i in year_list]) ]),
    #TASK 2.3: Add a division for output display
    html.Div( id='output-container', className='chart-grid', 
             style={'display':' flex'} ),
    html.Div( id='output2', className='output-div', 
             style={'display':' flex'}) 
])
# TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='select-year', component_property='disabled'),   # the return of your func.
    Input(component_id='stats',component_property='value')) # what you choose on the dash
def update_input_container(stat):
    if stat =='Yearly Statistics': 
        return False
    else: 
        return True

# def update_input_container(statistics):
#     if statistics ==:
#         message = f"You entered {statistics} "
#         return message  # Return selected_year and message
#     else:
#         message = "no statistics' type selected"
#         return dash.no_update
          

#Callback for plotting
#Define the callback function to update the input container based on the selected statistics

@app.callback(
    Output(component_id='output-container', component_property='children'),
    [Input(component_id='stats', component_property='value'),
     Input(component_id='select-year', component_property='value')])

def update_output_container(type,year):
    if type == 'Recession Period Statistics':
        # Filter the data for recession periods

        
        recession_data = data[data['Recession'] == 1]
        #TASK 2.5: Create and display graphs for Recession Report Statistics
        #Plot 1 Automobile sales fluctuate over Recession Period (year wise)
        # use groupby to create relevant data for plotting

        
        yearly_rec=recession_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        R_chart1 = dcc.Graph(figure=px.line(data_frame=yearly_rec, x='Year',y='Automobile_Sales',title="Average Automobile Sales fluctuation over Recession Period"))
        #Plot 2 Calculate the average number of vehicles sold by vehicle type       
        # use groupby to create relevant data for plotting

        
        average_sales = recession_data.groupby('Vehicle_Type')['Automobile_Sales'].mean().reset_index()                           
        R_chart2  = dcc.Graph(figure=px.bar(data_frame=average_sales,x='Vehicle_Type',y='Automobile_Sales'))
        # Plot 3 Pie chart for total expenditure share by vehicle type during recessions
        # use groupby to create relevant data for plotting

                              
        exp_rec= recession_data.groupby('Vehicle_Type')['Advertising_Expenditure'].sum()
        R_chart3 = dcc.Graph(figure=px.pie(exp_rec, values='Advertising_Expenditure', names=exp_rec.index))
        # Plot 4 bar chart for the effect of unemployment rate on vehicle type and sales

        
        unempl = recession_data.groupby('Vehicle_Type')[['unemployment_rate', 'Automobile_Sales']].mean().reset_index()                           
        R_chart4  = dcc.Graph(figure=px.bar(data_frame=unempl,x='Vehicle_Type',y=['Automobile_Sales', 'unemployment_rate'],barmode='group'))

                              
        return [html.Div(className='output-container', children=[html.Div(children=R_chart1),html.Div(children=R_chart2),html.Div(children=R_chart3),html.Div(children=R_chart4)],style={'display':' flex'})]
                          

    # TASK 2.6: Create and display graphs for Yearly Report Statistics
    # Yearly Statistic Report Plots                             
    elif (year and type=='Yearly Statistics') :
        yearly_data = data[data['Year'] == year]
        copy1= yearly_data            
        #TASK 2.5: Creating Graphs Yearly data
                              
        #plot 1 Yearly Automobile sales using line chart for the whole period.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(data_frame=yas, x='Year', y='Automobile_Sales',title= f"Avg. Yearly Automobile Sales fluctuation over {year}"))

        months=pd.to_datetime(yearly_data['Date']) 
        months=months.dt.month    
        copy1['months']=months
        copy1.groupby('months')[['Automobile_Sales']].sum()
        # Plot 2 Total Monthly Automobile sales using line chart.
        Y_chart2 = dcc.Graph(figure=px.line(data_frame=copy1,x=months,y='Automobile_Sales',title= f" Avg. Monthly Automobile Sales fluctuation over {year}"))

            # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=yearly_data.groupby('Vehicle_Type')[['Automobile_Sales']].mean().reset_index()
        Y_chart3 = dcc.Graph( figure=px.bar(data_frame=avr_vdata,x='Vehicle_Type',y='Automobile_Sales',title='Average Vehicles Sold by Vehicle Type in the year {}'.format(year)))

            # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby('Vehicle_Type')[['Advertising_Expenditure']].sum().reset_index()
        Y_chart4 = dcc.Graph(figure=px.pie(exp_data, values='Advertising_Expenditure',names=[x for x in exp_data['Vehicle_Type'].unique()],title='Advertising Expenditure by Vehicle Type' ))

        #TASK 2.6: Returning the graphs for displaying Yearly data
        return html.Div(className='output2', children=[html.Div(children=Y_chart1),html.Div(Y_chart2),html.Div(Y_chart3),html.Div(Y_chart4)],style={'textAlign':'left','color':'#008000', 'font-size':14})  
    else:
        return None


app.run_server(debug=True)


In [209]:
data.head(1)

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.5,27483.571,1558,7,60.223,0.01,5.4,456.0,Supperminicar,Georgia


In [238]:
copy1=data
months=pd.to_datetime(data['Date']) 
months=months.dt.month    
copy1['months']=months
data['Date'].head()
copy1

,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City,months
0,1/31/1980,1980,Jan,1,108.24,0.50,27483.571,1558,7,60.223,0.010000,5.4,456.0,Supperminicar,Georgia,1
1,2/29/1980,1980,Feb,1,98.75,0.75,24308.678,3048,4,45.986,-0.309594,4.8,555.9,Supperminicar,New York,2
2,3/31/1980,1980,Mar,1,107.48,0.20,28238.443,3137,3,35.141,-0.308614,3.4,620.0,Mediumfamilycar,New York,3
3,4/30/1980,1980,Apr,1,115.01,1.00,32615.149,1653,7,45.673,0.230596,4.2,702.8,Supperminicar,Illinois,4
4,5/31/1980,1980,May,1,98.72,0.20,23829.233,1319,4,52.997,0.138197,5.3,770.4,Smallfamiliycar,California,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,8/31/2023,2023,Aug,0,103.36,0.25,27852.993,1793,6,57.169,0.764155,2.6,1579.6,Executivecar,New York,8
524,9/30/2023,2023,Sep,0,101.55,0.07,21183.704,1028,5,59.315,0.036180,2.5,1123.4,Smallfamiliycar,California,9
525,10/31/2023,2023,Oct,0,124.66,0.12,15975.589,1148,9,19.472,-2.046169,2.5,1685.9,Sports,California,10
526,11/30/2023,2023,Nov,0,97.09,0.25,16862.288,4850,5,27.904,0.302179,2.9,2124.6,Smallfamiliycar,Georgia,11


In [220]:
data.head(1)


,Date,Year,Month,Recession,Consumer_Confidence,Seasonality_Weight,Price,Advertising_Expenditure,Competition,GDP,Growth_Rate,unemployment_rate,Automobile_Sales,Vehicle_Type,City
0,1/31/1980,1980,Jan,1,108.24,0.5,27483.571,1558,7,60.223,0.01,5.4,456.0,Supperminicar,Georgia


In [250]:
yearly_data = data[data['Year'] == 2000]
vehicle_types = [x for x in yearly_data['Vehicle_Type'].unique()]
print(vehicle_types)

['Supperminicar', 'Smallfamiliycar', 'Mediumfamilycar', 'Sports']


In [261]:
yearly_data = data[data['Year'] == 1980]
           
        #TASK 2.5: Creating Graphs Yearly data
                              
        #plot 1 Yearly Automobile sales using line chart for the whole period.
yas= yearly_data.groupby('Year')['Automobile_Sales'].mean().reset_index()
yas

,Year,Automobile_Sales
0,1980,678.95


In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px

# Load the data using pandas
data = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/historical_automobile_sales.csv')

# Initialize the Dash app
app = dash.Dash(__name__)

# Set the title of the dashboard
#app.title = "Automobile Statistics Dashboard"

#---------------------------------------------------------------------------------
# Create the dropdown menu options
dropdown_options = [
    {'label': '...........', 'value': 'Yearly Statistics'},
    {'label': 'Recession Period Statistics', 'value': '.........'}
]
# List of years 
year_list = [i for i in range(1980, 2024, 1)]
#---------------------------------------------------------------------------------------
# Create the layout of the app
app.layout = html.Div([
    #TASK 2.1 Add title to the dashboard
    html.H1("..............."),#May include style for title
    html.Div([#TASK 2.2: Add two dropdown menus
        html.Label("Select Statistics:"),
        dcc.Dropdown(
            id='...........',
            options=...................,
            value='.................',
            placeholder='.................'
        )
    ]),
    html.Div(dcc.Dropdown(
            id='select-year',
            options=[{'label': i, 'value': i} for i in year_list],
            value='...................'
        )),
    html.Div([#TASK 2.3: Add a division for output display
    html.Div(id='..........', className='..........', style={.........}),])
])
#TASK 2.4: Creating Callbacks
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='......', component_property='....'),
    Input(component_id='..........',component_property='....'))

def update_input_container(.......):
    if selected_statistics =='........': 
        return False
    else: 
        return ......

#Callback for plotting
# Define the callback function to update the input container based on the selected statistics
@app.callback(
    Output(component_id='...', component_property='...'),
    [Input(component_id='...', component_property='...'), Input(component_id='...', component_property='...')])


def update_output_container(....., .....):
    if ..... == 'Recession Period Statistics':
        # Filter the data for recession periods
        recession_data = data[data['Recession'] == 1]
        
#TASK 2.5: Create and display graphs for Recession Report Statistics

#Plot 1 Automobile sales fluctuate over Recession Period (year wise)
        # use groupby to create relevant data for plotting
        yearly_rec=recession_data.groupby('...')['...'].mean().reset_index()
        R_chart1 = dcc.Graph(
            figure=px......(....., 
                x='....',
                y='......',
                title="Average Automobile Sales fluctuation over Recession Period"))

#Plot 2 Calculate the average number of vehicles sold by vehicle type       
        # use groupby to create relevant data for plotting
        average_sales = ...............mean().reset_index()                           
        R_chart2  = dcc.Graph(figure=px....................
        
# Plot 3 Pie chart for total expenditure share by vehicle type during recessions
        # use groupby to create relevant data for plotting
        exp_rec= ....................
        R_chart3 = .............

# Plot 4 bar chart for the effect of unemployment rate on vehicle type and sales
        ................
        ...................


        return [
            html.Div(className='..........', children=[html.Div(children=R_chart1),html.Div(children=.....)],style={.....}),
            html.Div(className='chart-item', children=[html.Div(children=...........),html.Div(.............)],style={....})
            ]

# TASK 2.6: Create and display graphs for Yearly Report Statistics
 # Yearly Statistic Report Plots                             
    elif (input_year and selected_statistics=='...............') :
        yearly_data = data[data['Year'] == ......]
                              
#TASK 2.5: Creating Graphs Yearly data
                              
#plot 1 Yearly Automobile sales using line chart for the whole period.
        yas= data.groupby('Year')['Automobile_Sales'].mean().reset_index()
        Y_chart1 = dcc.Graph(figure=px.line(.................))
            
# Plot 2 Total Monthly Automobile sales using line chart.
        Y_chart2 = dcc.Graph(................)

            # Plot bar chart for average number of vehicles sold during the given year
        avr_vdata=yearly_data.groupby........................
        Y_chart3 = dcc.Graph( figure.................,title='Average Vehicles Sold by Vehicle Type in the year {}'.format(input_year)))

            # Total Advertisement Expenditure for each vehicle using pie chart
        exp_data=yearly_data.groupby(..................
        Y_chart4 = dcc.Graph(...............)

#TASK 2.6: Returning the graphs for displaying Yearly data
        return [
                html.Div(className='.........', children=[html.Div(....,html.Div(....)],style={...}),
                html.Div(className='.........', children=[html.Div(....),html.Div(....)],style={...})
                ]
        
    else:
        return None

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

